In [1]:
import pandas as pd

import wmfdata as wmf

I just ran a test version of a changed data aggreggation job ([T297173](https://phabricator.wikimedia.org/T297173)). Now I need to check that the data generated by the test version (`new_data`) matches the data generated by the existing version (`old_data`). The only difference should be the new version classifies some previews as having `access_method = 'mobile web'`.

In [9]:
old_data = wmf.presto.run("""
SELECT *
FROM wmf_product.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 1
    AND day BETWEEN 2 AND 7
""")

new_data = wmf.presto.run("""
SELECT *
FROM nshahquinn.wikipediapreview_stats
WHERE
    year = 2022
    AND month = 1
    AND day BETWEEN 2 AND 7
""")

In [12]:
old_data.groupby(["year", "month", "day"])[["pageviews", "previews"]].sum()

pageviews  previews
year month day                     
2022 1     2           29       317
           3           26       311
           4           31       577
           5           28       581
           6           31       469
           7           27       424

In [13]:
new_data.groupby(["year", "month", "day"])[["pageviews", "previews"]].sum()

pageviews  previews
year month day                     
2022 1     2           29       317
           3           26       311
           4           31       577
           5           28       581
           6           31       469
           7           27       424

In [19]:
(
    old_data
    .query("access_method != 'mobile app'")
    .groupby(["year", "month", "day", "access_method"])
    ["previews"]
    .sum()
    .unstack()
    .assign(total=lambda df: df["desktop"] + df["mobile web"])
)

access_method   desktop  mobile web  total
year month day                            
2022 1     2        317           0    317
           3        311           0    311
           4        577           0    577
           5        578           2    580
           6        469           0    469
           7        424           0    424

In [20]:
(
    new_data
    .query("access_method != 'mobile app'")
    .groupby(["year", "month", "day", "access_method"])
    ["previews"]
    .sum()
    .unstack()
    .assign(total=lambda df: df["desktop"] + df["mobile web"])
)

access_method   desktop  mobile web  total
year month day                            
2022 1     2        317           0    317
           3        301          10    311
           4        547          30    577
           5        576           4    580
           6        466           3    469
           7        424           0    424

Everything looks good!